In [1]:
!pip install -q transformers datasets librosa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from sklearn import metrics

In [3]:
# !unzip 'disgust-20241215T174917Z-001.zip'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
valid_data = pd.read_csv("/content/drive/MyDrive/Проект/test_without_answers.csv")

In [6]:
# для теста пока первые 5 строк
#valid_data = valid_data[:5]
valid_data.head()

,text,anger,disgust,fear,joy,sadness,surprise,neutral
0,надеюсь вы сохранили это письмо,0,0,0,0,0,0,0
1,я зашел в ваш профиль чтобы посмотреть другие ...,0,0,0,0,0,0,0
2,ты реально пустое место,0,0,0,0,0,0,0
3,я не могу сказать счастливый это конец или гру...,0,0,0,0,0,0,0
4,это 8 раундов по разным темам организованных п...,0,0,0,0,0,0,0


In [7]:
emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
#emotion_labels = ["anger" , "disgust"]

In [8]:
# Функция для предсказания эмоций
def predict_emotions(texts):
    emotion_labels = ['not' + emotion, emotion]
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)

    # Преобразуем вероятности в словарь
    return [{emotion: prob.item() for emotion, prob in zip(emotion_labels, probabilities[i])} for i in range(probabilities.size(0))]

In [9]:
# Загрузка токенизатора и модели
tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [10]:
# создаем нулевой датасет для заполнения
df_valid = valid_data[['text']].copy()

for emotion in emotion_labels:
  df_valid[emotion] = 0

df_valid

,text,anger,disgust,fear,joy,sadness,surprise,neutral
0,надеюсь вы сохранили это письмо,0,0,0,0,0,0,0
1,я зашел в ваш профиль чтобы посмотреть другие ...,0,0,0,0,0,0,0
2,ты реально пустое место,0,0,0,0,0,0,0
3,я не могу сказать счастливый это конец или гру...,0,0,0,0,0,0,0
4,это 8 раундов по разным темам организованных п...,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
8737,просто устал от всего этого если честно,0,0,0,0,0,0,0
8738,да она была но я сейчас здесь,0,0,0,0,0,0,0
8739,это случается со мной часто я тоже буду петь о...,0,0,0,0,0,0,0
8740,это был друг редактирования они один и тот же ...,0,0,0,0,0,0,0


In [11]:
for emotion in emotion_labels:
  model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Проект/' + emotion + '/')

  # Получение предсказаний на тестовой выборке
  predictions = predict_emotions(valid_data.text.to_list())

  df_tmp=pd.DataFrame(predictions)
  df_tmp.rename(columns={emotion: emotion + '_proba'}, inplace=True)

  df_tmp[emotion] = df_tmp[ emotion + '_proba'].apply(lambda x: 1 if x >= 0.5 else 0)

  result = pd.concat([valid_data['text'], df_tmp[[emotion + '_proba', emotion]]], axis=1)


  # Вывод предсказаний
  #for text, pred in zip(valid_data.text.to_list(), predictions):
  #  print(f"Text: {text}, Predictions: {pred}")

df_valid[emotion]=result[emotion]
df_valid

,text,anger,disgust,fear,joy,sadness,surprise,neutral
0,надеюсь вы сохранили это письмо,0,0,0,0,0,0,0
1,я зашел в ваш профиль чтобы посмотреть другие ...,0,0,0,0,0,0,0
2,ты реально пустое место,0,0,0,0,0,0,0
3,я не могу сказать счастливый это конец или гру...,0,0,0,0,0,0,0
4,это 8 раундов по разным темам организованных п...,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...
8737,просто устал от всего этого если честно,0,0,0,0,0,0,0
8738,да она была но я сейчас здесь,0,0,0,0,0,0,1
8739,это случается со мной часто я тоже буду петь о...,0,0,0,0,0,0,0
8740,это был друг редактирования они один и тот же ...,0,0,0,0,0,0,1


In [13]:
df_valid['id'] = range(1, len(df_valid) + 1)
df_valid = df_valid.drop(columns=['text'])

In [14]:
df_valid.to_csv("/content/drive/MyDrive/Проект/submission.csv", index=False)